# Config

In [2]:
app_id = "ZYNNQ1EECF-100"
secret_id = "3W4F0X0G11"
redirect_url = "https://www.google.com/"

market_opening_hour = 9
market_opening_minute = 15
market_closing_hour = 15
market_closing_minute = 15
start_date = "2023-06-17" 
end_date = "2023-09-25"

fetch_data_from = "csv"

# Fyers Client Session

In [3]:
from fyers_apiv3 import fyersModel

client_id = app_id
secret_key = secret_id
redirect_uri = redirect_url
response_type = "code"  

session = fyersModel.SessionModel(
    client_id=app_id,
    secret_key=secret_id,
    redirect_uri=redirect_url,
    response_type="code",
    grant_type = "authorization_code"
)

response = session.generate_authcode()

print(response)

https://api-t1.fyers.in/api/v3/generate-authcode?client_id=ZYNNQ1EECF-100&redirect_uri=https%3A%2F%2Fwww.google.com%2F&response_type=code&state=None


# Auth Code From Redirect Url

In [4]:
auth_code="eyJ0eXAiOiJKV1QiLCJhbGciOiJIUzI1NiJ9.eyJpc3MiOiJhcGkubG9naW4uZnllcnMuaW4iLCJpYXQiOjE2OTY2NzUxODgsImV4cCI6MTY5NjcwNTE4OCwibmJmIjoxNjk2Njc0NTg4LCJhdWQiOiJbXCJ4OjBcIiwgXCJ4OjFcIiwgXCJ4OjJcIiwgXCJkOjFcIiwgXCJkOjJcIiwgXCJ4OjFcIiwgXCJ4OjBcIl0iLCJzdWIiOiJhdXRoX2NvZGUiLCJkaXNwbGF5X25hbWUiOiJYTDA1NzM0Iiwib21zIjoiSzEiLCJoc21fa2V5IjoiZWQxNGYwNTE1N2JkOGQ1NTQ1Y2IyNmZiODAxZTI0NDBlZGYyNmZiYjkzZjQ4ZmVmOGFiZmUxYWQiLCJub25jZSI6IiIsImFwcF9pZCI6IlpZTk5RMUVFQ0YiLCJ1dWlkIjoiYzVkMWI3ZWQ1NTBhNDhhODk4M2NhZGNjYjk2MTljMDYiLCJpcEFkZHIiOiIwLjAuMC4wIiwic2NvcGUiOiIifQ.E2XIkBwmLijJlymWjPrh0flMJQUBHfLmQUvKX_X8IiA"

# Session Status and Access Token

In [5]:
session.set_token(auth_code)

response = session.generate_token()

access_token = response['access_token']

print(response)

{'s': 'ok', 'code': 200, 'message': '', 'access_token': 'eyJ0eXAiOiJKV1QiLCJhbGciOiJIUzI1NiJ9.eyJpc3MiOiJhcGkuZnllcnMuaW4iLCJpYXQiOjE2OTY2NzUyMTMsImV4cCI6MTY5NjcyNTAxMywibmJmIjoxNjk2Njc1MjEzLCJhdWQiOlsieDowIiwieDoxIiwieDoyIiwiZDoxIiwiZDoyIiwieDoxIiwieDowIl0sInN1YiI6ImFjY2Vzc190b2tlbiIsImF0X2hhc2giOiJnQUFBQUFCbElUV04tYWtWYkpva2NnMHdxMVRzWWhBUnJ6YTZyMkZsQTNiQ2JxVWNqUzdKZElDcXlhQmJ5UTNUTndIUk5zelpYVXEtX25GekRUYWdrbVI4dUJ3bHAtVDdSc0psQjktc3djSDNuX04zdVFxRS11az0iLCJkaXNwbGF5X25hbWUiOiJMQUtTSElUIiwib21zIjoiSzEiLCJoc21fa2V5IjoiZWQxNGYwNTE1N2JkOGQ1NTQ1Y2IyNmZiODAxZTI0NDBlZGYyNmZiYjkzZjQ4ZmVmOGFiZmUxYWQiLCJmeV9pZCI6IlhMMDU3MzQiLCJhcHBUeXBlIjoxMDAsInBvYV9mbGFnIjoiTiJ9.Rq_WwSsoIqkO_PWIcUvc4Cage81g_Pobo_Zco0dRfeY', 'refresh_token': 'eyJ0eXAiOiJKV1QiLCJhbGciOiJIUzI1NiJ9.eyJpc3MiOiJhcGkuZnllcnMuaW4iLCJpYXQiOjE2OTY2NzUyMTMsImV4cCI6MTY5NzkzNDYxMywibmJmIjoxNjk2Njc1MjEzLCJhdWQiOlsieDowIiwieDoxIiwieDoyIiwiZDoxIiwiZDoyIiwieDoxIiwieDowIl0sInN1YiI6InJlZnJlc2hfdG9rZW4iLCJhdF9oYXNoIjoiZ0FBQUFBQmxJVFdOLWFrVmJK

# Create Fyers Client

In [27]:
from os import getcwd
client = fyersModel.FyersModel(client_id=client_id, is_async=False, token=access_token, log_path=getcwd())

# Historical Data

## Fyers

In [28]:
import pandas as pd
import pytz

if fetch_data_from == "fyers":
    data = {
        "symbol":"NSE:NIFTYBANK-INDEX",
        "resolution":"15",
        "date_format":"1",
        "range_from": start_date,
        "range_to": end_date,
        "cont_flag":"1"
    }

    response = client.history(data=data)
    # print(response)
    df = pd.DataFrame(response['candles'], columns=['Time', "Open", "High", "Low", "Close", "Volume"])    

    # Epoch Time to Human Readable
    ist = pytz.timezone('Asia/Kolkata')
    df['Time'] = pd.to_datetime(df['Time'], unit='s').dt.tz_localize(pytz.UTC).dt.tz_convert(ist)   

    df = df.set_index("Time") 

## Csv Import

In [29]:
if fetch_data_from == "csv":
    df = pd.read_csv('NiftyData.csv')
    df = df.drop(columns=["Unnamed: 0", "INDEX_NAME", "Index Name"])
    df.rename(columns={"HistoricalDate":"Time", "OPEN": "Open", "LOW": "Low", "CLOSE":"Close", "HIGH": "High" }, inplace=True)
    df = df[::-1].reset_index(drop=True)

In [30]:
df.head()

,Time,Open,High,Low,Close
0,03 Jan 2000,1482.15,1592.90,1482.15,1592.2
1,04 Jan 2000,1594.40,1641.95,1594.40,1638.7
2,05 Jan 2000,1634.55,1635.50,1555.05,1595.8
3,06 Jan 2000,1595.80,1639.00,1595.80,1617.6
4,07 Jan 2000,1616.60,1628.25,1597.20,1613.3


# Time Difference in Months

In [31]:
from datetime import datetime

if fetch_data_from == "fyers":
    start_date_ = datetime.strptime(start_date, "%Y-%m-%d")
    end_date_ = datetime.strptime(end_date, "%Y-%m-%d")

elif fetch_data_from == "csv":
    start_date_ = pd.to_datetime(df['Time'][0])
    end_date_ = pd.to_datetime(df["Time"][-1:].values)
    start_date = df['Time'].head(1).values[0]
    end_date = df['Time'].tail(1).values[0]
months_difference = ((end_date_.year - start_date_.year) * 12 + (end_date_.month - start_date_.month))[0]


# Change in Price

In [58]:
period = 2

from numpy import nan
from math import isnan

df[f'{period}_day_price_change'] = df['Close'].diff(periods=period)
prediction = []
for i in range(0, len(df)):
    is_positive_trend = df[f'{period}_day_price_change'].iloc[i] > 0
    if is_positive_trend and not isnan(df[f'{period}_day_price_change'].iloc[i]):
       prediction.append("Gap Up")
    elif not is_positive_trend and not isnan(df[f'{period}_day_price_change'].iloc[i]):
        prediction.append('Gap Down')
    elif isnan(df[f'{period}_day_price_change'].iloc[i]):
        prediction.append(nan)
df['Change_Price_Prediction'] = prediction

## Actual Gap

In [67]:
gap_list = []
for i in range(0, len(df) - 1):
    gap = df.iloc[i+1].Open - df.iloc[i].Close
    gap_list.append(round(gap,2))    
df.loc[0, 'Actual_Gap'] = nan
df['Actual_Gap'].iloc[period-1:] =  gap_list    

/var/folders/7k/mwk0ry_90p1gvfnc7vymq21w0000gn/T/ipykernel_7398/1635259681.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Actual_Gap'].iloc[period-1:] =  gap_list


## Historical Trend with SMA

In [63]:
short_window = 5
long_window = 20

df['SMA_Short'] = df['Close'].rolling(window=short_window).mean()
df['SMA_Long'] = df['Close'].rolling(window=long_window).mean()
sma_prediction = []
for i in range(0, len(df)):
    if df['SMA_Short'].iloc[i] > df['SMA_Long'].iloc[i]:
        sma_prediction.append("Up Trend")
    elif df['SMA_Short'].iloc[i] < df['SMA_Long'].iloc[i]:
        sma_prediction.append("Down Trend")    

df['Historical_Trend'] = nan
df['Historical_Trend'].iloc[long_window:] = sma_prediction

/var/folders/7k/mwk0ry_90p1gvfnc7vymq21w0000gn/T/ipykernel_7398/3569904789.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Historical_Trend'].iloc[long_window:] = sma_prediction


# Day Opening and Closing Data Collection

In [12]:
if fetch_data_from == "fyers":
    trade_info = {
        'day_closing': [],
        'day_opening': []
    }
    for i in range(0,len(df)):
        if df.index[i].hour == market_opening_hour and df.index[i].minute == market_opening_minute:
                trade_info['day_opening'].append(df.index[i])
        elif df.index[i].hour == market_closing_hour and df.index[i].minute == market_closing_minute:
                trade_info['day_closing'].append(df.index[i])

In [69]:
df

,Time,Open,High,Low,Close,2_day_price_change,Change_Price_Prediction,SMA_Short,SMA_Long,Historical_Trend,Actual_Gap
0,03 Jan 2000,1482.15,1592.90,1482.15,1592.20,NaN,NaN,NaN,NaN,NaN,NaN
1,04 Jan 2000,1594.40,1641.95,1594.40,1638.70,NaN,NaN,NaN,NaN,NaN,2.20
2,05 Jan 2000,1634.55,1635.50,1555.05,1595.80,3.60,Gap Up,NaN,NaN,NaN,-4.15
3,06 Jan 2000,1595.80,1639.00,1595.80,1617.60,-21.10,Gap Down,NaN,NaN,NaN,0.00
4,07 Jan 2000,1616.60,1628.25,1597.20,1613.30,17.50,Gap Up,1611.52,NaN,NaN,-1.00
...,...,...,...,...,...,...,...,...,...,...,...
5810,15 May 2023,18339.30,18458.90,18287.90,18398.85,101.85,Gap Up,18318.34,17997.9475,Up Trend,24.50
5811,16 May 2023,18432.35,18432.35,18264.35,18286.50,-28.30,Gap Down,18322.45,18026.9300,Up Trend,33.50
5812,17 May 2023,18300.45,18309.00,18115.35,18181.75,-217.10,Gap Down,18295.78,18053.0100,Up Trend,13.95
5813,18 May 2023,18287.50,18297.20,18104.85,18129.95,-156.55,Gap Down,18262.37,18078.5700,Up Trend,105.75


# BTST Logic

In [13]:
pnl_list = []

if fetch_data_from == "fyers":
    for i in range(0, len(trade_info['day_closing']) - 1):
        pnl = df.loc[trade_info['day_opening'][i+1]].Open - df.loc[trade_info['day_closing'][i]].Open
        pnl_list.append(round(pnl,2))

if fetch_data_from == "csv":
    for i in range(0, len(df) - 1):
        pnl = df.iloc[i+1].Open - df.iloc[i].Close
        pnl_list.append(round(pnl,2))    

# BTST Results

In [14]:
total_profits = sum(1 for profit in pnl_list if profit > 0)
total_losses = sum(1 for loss in pnl_list if loss < 0)

In [15]:
print(total_profits)
print(total_losses)

3726
1947


## Paper Trading Results

In [16]:
def ConsecutivePL(pnl_data):
    l_streak, w_streak = 0, 0
    max_loosing_streak, max_winning_streak = 0,0
    for i in pnl_data:
        if i < 0:
            l_streak+=1
            if l_streak > max_loosing_streak:
                max_loosing_streak = l_streak        
        elif i > 0:
            l_streak = 0
        if i > 0:
            w_streak+=1
            if w_streak > max_winning_streak:
                max_winning_streak = w_streak        
        elif i < 0:
            w_streak = 0
    return {"Winning_Streak": max_winning_streak, "Loosing_Streak": max_loosing_streak }

def MaxConsecutivePL(pnl_data, lot_size, no_of_lots):
    l_streak, p_streak = 0, 0
    max_loss_streak, max_profit_streak = 0,0
    for i in pnl_data:
        if i < 0:
            l_streak+=i
            if l_streak < max_loss_streak:
                max_loss_streak = l_streak        
        elif i > 0:
            l_streak = 0
        if i > 0:
            p_streak+=i
            if p_streak > max_profit_streak:
                max_profit_streak = p_streak        
        elif i < 0:
            p_streak = 0
    return {"Max_Profit_Streak": round(max_profit_streak*lot_size*no_of_lots,2), 
            "Max_Loss_Streak": round(max_loss_streak*lot_size*no_of_lots,2) }

def WLRate(pnl_data):
    win_rate = 0
    loss_rate = 0
    for pnl in pnl_data:
        if pnl > 0:
            win_rate+=1
        else:
            loss_rate+=1
    total = total_profits + total_losses
    win_rate = round(total_profits*100/total,2)
    loss_rate = round(total_losses*100/total,2)
    return {"Win_Rate": win_rate, "Loss_Rate": loss_rate}


# Max DrawDown
def MaxDD(capital_movement):
    max_drawdown = 0
    current_drawdown = 0
    peak_value = capital_movement[0]
    for capital in capital_movement:
        if capital > peak_value:
            peak_value = capital

        current_drawdown = peak_value - capital
        if current_drawdown > max_drawdown:
            max_drawdown = current_drawdown
    return max_drawdown

def FinalCapital(inital_capital, no_of_lots, lot_size):
    final_capital = inital_capital
    capital_movement = [final_capital]
    for pnl in pnl_list:
        final_capital += (pnl*no_of_lots*lot_size)
        capital_movement.append(final_capital) 
    return final_capital, capital_movement

def PNLs(no_of_lots, lot_size):
    pnls = []
    for pnl in pnl_list:
        pnls.append(pnl*no_of_lots*lot_size)
    return pnls

def AveragePNL(pnl_data,no_of_lots,lot_size):
    total_sum = 0
    for pnl in pnl_data:
        total_sum+=(pnl*no_of_lots*lot_size)
    average = total_sum/len(pnl_data)
    return round(average,2)

def AveragePNLPerTrade(pnl_data, no_of_lots,lot_size):
    total_sum_l,total_sum_p = 0,0
    no_of_losses, no_of_profits = 0,0
    for pnl in pnl_data:
        if pnl < 0:
            total_sum_l+=(pnl*no_of_lots*lot_size)
            no_of_losses+=1
        elif pnl > 0:
            total_sum_p+=(pnl*no_of_lots*lot_size)
            no_of_profits+=1
    average_loss = round(total_sum_l/no_of_losses,2)
    average_profit = round(total_sum_p/no_of_profits,2)
    return {"Average_Profits_Per_Trade": average_profit, "Average_Loss_Per_Trade": average_loss}

In [17]:
inital_capital = 200000
no_of_lots = 2
lot_size = 50
capital_movement = FinalCapital(inital_capital, no_of_lots, lot_size)[1]

final_capital = FinalCapital(inital_capital, no_of_lots, lot_size)[0]
roi = final_capital - inital_capital
roi_per = roi*100/inital_capital
max_profit = round(max(PNLs(no_of_lots, lot_size)),2)
max_loss = min(PNLs(no_of_lots, lot_size))
max_drawdown = MaxDD(capital_movement)
mdd_per = round(max_drawdown*100/inital_capital,2)
max_winning_streak = ConsecutivePL(pnl_list)['Winning_Streak']
max_loosing_streak = ConsecutivePL(pnl_list)['Loosing_Streak']
pnl_per_trade = AveragePNL(pnl_list,no_of_lots,lot_size)
max_profit_steak = MaxConsecutivePL(pnl_list, lot_size, no_of_lots)['Max_Profit_Streak']
max_loss_steak = MaxConsecutivePL(pnl_list,lot_size, no_of_lots)['Max_Loss_Streak']
win_rate = WLRate(pnl_list)['Win_Rate']
loss_rate = WLRate(pnl_list)['Loss_Rate']
average_profit_on_win_trade = AveragePNLPerTrade(pnl_list, no_of_lots,lot_size)["Average_Profits_Per_Trade"]
average_loss_on_loss_trade = AveragePNLPerTrade(pnl_list, no_of_lots,lot_size)["Average_Loss_Per_Trade"]
return_over_mdd = round(roi_per/mdd_per,2)
risk_to_reward = round(abs(average_profit_on_win_trade/average_loss_on_loss_trade),2)
expectancy_ratio = round((risk_to_reward*(win_rate/100))-(loss_rate/100),2)

In [18]:
from IPython.display import Markdown as md

md("# Performance Summary\n Start Date: {}</br> End Date: {}</br>Time Period: {} Months</br> Initial Capital: {}</br>\
Final Capital: {}</br> \
   Return on Investment: {}</br> \
   ROI Percentage: {}%</br> Max Profit: {}</br> Max Loss: {}</br> \
  Max Draw Down: {}</br> Max Draw Down Percentage: {} </br> Max Winning Streak: {} \
  </br> Max Loosing Streak: {}</br> Average PNL Per Trade: {}</br>\
  Max Profit Streak: {} </br> Max Loss Streak: {}</br> Win Rate: {}% </br> Loss Rate: {}%</br>\
  Average Profit on Win Trade: {}</br> Average Loss on Loss Trade: {}</br>\
  Return Over Max Draw Down: {}</br> Risk to Reward Ratio: {}</br> Expectancy Ratio: {}".format(
                                    start_date, end_date, months_difference, inital_capital,
                                    final_capital, roi, roi_per,
                                    max_profit,max_loss, max_drawdown,
                                    mdd_per,max_winning_streak, max_loosing_streak, pnl_per_trade,
                                    max_profit_steak, max_loss_steak, win_rate, loss_rate,
                                    average_profit_on_win_trade, average_loss_on_loss_trade,
                                    return_over_mdd, risk_to_reward, expectancy_ratio))

# Performance Summary
 Start Date: 03 Jan 2000</br> End Date: 19 May 2023</br>Time Period: 280 Months</br> Initial Capital: 200000</br>Final Capital: 4053330.0</br>    Return on Investment: 3853330.0</br>    ROI Percentage: 1926.665%</br> Max Profit: 41175.0</br> Max Loss: -79975.0</br>   Max Draw Down: 293415.0</br> Max Draw Down Percentage: 146.71 </br> Max Winning Streak: 23   </br> Max Loosing Streak: 7</br> Average PNL Per Trade: 662.77</br>  Max Profit Streak: 86485.0 </br> Max Loss Streak: -195930.0</br> Win Rate: 65.68% </br> Loss Rate: 34.32%</br>  Average Profit on Win Trade: 2302.36</br> Average Loss on Loss Trade: -2426.95</br>  Return Over Max Draw Down: 13.13</br> Risk to Reward Ratio: 0.95</br> Expectancy Ratio: 0.28

# Historical Trend 

## Predicting Gaps

#### Actual Gap

In [20]:
df.loc[0, 'Actual_Gap'] = nan
df['Actual_Gap'].iloc[period-1:] =  pnl_list

/var/folders/7k/mwk0ry_90p1gvfnc7vymq21w0000gn/T/ipykernel_7398/3094274696.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Actual_Gap'].iloc[period-1:] =  pnl_list


### Evluation

In [22]:
wins = 0
losses = 0
for i in range(0, len(df)):
    if (df['Actual_Gap'].iloc[i] > 0 and df["Change_Price_Prediction"].iloc[i] == "Gap Up" and df['Historical_Trend'].iloc[i] == "Up Trend") or (df['Actual_Gap'].iloc[i] < 0 and df["Change_Price_Prediction"].iloc[i] == "Gap Down" and df['Historical_Trend'].iloc[i] == "Down Trend"):
        wins+=1
    elif (df['Actual_Gap'].iloc[i] < 0 and df["Change_Price_Prediction"].iloc[i] == "Gap Up" and df['Historical_Trend'].iloc[i] == "Up Trend") or (df['Actual_Gap'].iloc[i] > 0 and df["Change_Price_Prediction"].iloc[i] == "Gap Down" and df['Historical_Trend'].iloc[i] == "Down Trend"):
        losses+=1    
    


In [23]:
win_rate__ = round(wins*100/(wins+losses),2)

In [24]:
print(win_rate__,"%")

66.44 %
